In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/submission-merged3/submission_lightgbm.csv
/kaggle/input/submission-merged3/submission_catboost.csv
/kaggle/input/submission-merged3/submission_merged3.csv
/kaggle/input/submission-merged3/catboost_info/learn_error.tsv
/kaggle/input/submission-merged3/catboost_info/catboost_training.json
/kaggle/input/submission-merged3/catboost_info/time_left.tsv
/kaggle/input/submission-merged3/catboost_info/learn/events.out.tfevents
/kaggle/input/tabular-playground-series-apr-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-apr-2021/train.csv
/kaggle/input/tabular-playground-series-apr-2021/test.csv


In [2]:
!pip install scikit-learn==0.23.2

     |████████████████████████████████| 6.8 MB 4.8 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.1
    Uninstalling scikit-learn-0.24.1:
      Successfully uninstalled scikit-learn-0.24.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyldavis 3.3.1 requires numpy>=1.20.0, but you have numpy 1.19.5 which is incompatible.
pdpbox 0.2.1 requires matplotlib==3.1.1, but you have matplotlib 3.4.0 which is incompatible.
imbalanced-learn 0.8.0 requires scikit-learn>=0.24, but you have scikit-learn 0.23.2 which is incompatible.


https://www.kaggle.com/udbhavpangotra/tps-apr21-eda-model


https://www.kaggle.com/hiro5299834/tps-apr-2021-voting-pseudo-labeling

# KAGGLE 스터디 

In [3]:
import pandas as pd
import numpy as np
import random
import os

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold

import lightgbm as lgb
import catboost as ctb
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier, export_graphviz

import graphviz
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter('ignore')

In [4]:
TARGET = 'Survived'

N_ESTIMATORS = 1000
N_SPLITS = 10
SEED = 2021
EARLY_STOPPING_ROUNDS = 100
VERBOSE = 100

In [5]:
# #랜덤 시드 생성
def set_seed(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

## 데이터 전처리

### lode data

In [6]:
train_df = pd.read_csv('../input/tabular-playground-series-apr-2021/train.csv')
test_df = pd.read_csv('../input/tabular-playground-series-apr-2021/test.csv')
submission = pd.read_csv('../input/tabular-playground-series-apr-2021/sample_submission.csv')
#test_df['Survived'] = pd.read_csv("../input/submission-merged3/submission_merged3.csv")['Survived']

all_df = pd.concat([train_df, test_df]).reset_index(drop=True)
#reset_index : 인덱스를 세팅한다. drop=True를 하면 인덱스를 세팅한걸 삭제함. 


In [7]:
print('Rows and Columns in train dataset:', train_df.shape)
print('Rows and Columns in test dataset:', test_df.shape)

Rows and Columns in train dataset: (100000, 12)
Rows and Columns in test dataset: (100000, 11)


### 결측치 갯수 출력

In [8]:
print('Missing values per columns in train dataset')
for col in train_df.columns:
    temp_col = train_df[col].isnull().sum()
    print(f'{col}: {temp_col}')
print()
print('Missing values per columns in test dataset')
for col in test_df.columns:
    temp_col = test_df[col].isnull().sum()
    print(f'{col}: {temp_col}')

Missing values per columns in train dataset
PassengerId: 0
Survived: 0
Pclass: 0
Name: 0
Sex: 0
Age: 3292
SibSp: 0
Parch: 0
Ticket: 4623
Fare: 134
Cabin: 67866
Embarked: 250

Missing values per columns in test dataset
PassengerId: 0
Pclass: 0
Name: 0
Sex: 0
Age: 3487
SibSp: 0
Parch: 0
Ticket: 5181
Fare: 133
Cabin: 70831
Embarked: 277


### Filling missing values

In [9]:
#나이는 나이의 평균치로 채운다.
all_df['Age'] = all_df['Age'].fillna(all_df['Age'].mean())

#cabin은 문자열을 분할하고, 제일 첫번째 글자를 따와서 넣는다. 결측치엔 X를 넣는다.
#strip() : 양쪽 공백을 지운다. 여기서느 x[0]외엔 다 지우는듯. 
all_df['Cabin'] = all_df['Cabin'].fillna('X').map(lambda x: x[0].strip())


#print(all_df['Ticket'].head(10))
#Ticket, fillna with 'X', split string and take first split 
#split() : 문자열 나누기. 디폴트는 ' '이고, 문자를 가진 데이터들이 전부 띄워쓰기로 구분되어있기때문에 가능. 
all_df['Ticket'] = all_df['Ticket'].fillna('X').map(lambda x:str(x).split()[0] if len(str(x).split()) > 1 else 'X')

#pclass에 따른 Fare의 평균을 구해서 dictionary형태로 만든다. 
fare_map = all_df[['Fare', 'Pclass']].dropna().groupby('Pclass').median().to_dict()
#fare의 결측치에 본인 행의 pclass 값을 넣고, 그 값을 fare 평균에 맵핑시킨다.  
all_df['Fare'] = all_df['Fare'].fillna(all_df['Pclass'].map(fare_map['Fare']))
#유독 높은 가격이나 낮은 가격이 있기때문에, 이상치의 영향을 줄이기 위해서 Fare에 log를 취해준다.
all_df['Fare'] = np.log1p(all_df['Fare'])


#항구의 결측치를 X로 채운다. 
all_df['Embarked'] = all_df['Embarked'].fillna('X')

#이름은 성만 사용한다.
all_df['Name'] = all_df['Name'].map(lambda x: x.split(',')[0])


In [10]:
data_1=all_df.loc[all_df['Pclass']==1].groupby('Ticket')['Ticket'].count().sort_values(ascending=False)
print(data_1)
print()
data_2=all_df.loc[all_df['Pclass']==2].groupby('Ticket')['Ticket'].count().sort_values(ascending=False)
print(data_2)
print()
data_3=all_df.loc[all_df['Pclass']==3].groupby('Ticket')['Ticket'].count().sort_values(ascending=False)
print(data_3)
print()

Ticket
X             36336
PC            16814
C.A.            338
SC/Paris        334
SC/PARIS        260
W./C.           206
S.O.C.          192
S.C./PARIS      191
PP              186
F.C.            183
SC/AH           178
F.C.C.          167
STON/O          163
CA.             161
SOTON/O.Q.      123
A/4             115
A/5.            108
W.E.P.           94
WE/P             92
SOTON/OQ         87
CA               81
STON/O2.         81
A/5              70
C                67
A/4.             66
P/PP             66
SC               59
SOTON/O2         48
A./5.            46
S.O./P.P.        40
A.5.             33
AQ/4             27
A/S              23
SCO/W            19
S.P.             17
SC/A4            16
SW/PP            16
SC/A.3           15
S.O.P.           15
C.A./SOTON       14
A.               14
SO/C             14
S.C./A.4.        14
STON/OQ.         13
W/C              13
LP               11
S.W./PP          11
AQ/3.             8
Fa                7
A4.          

## 인코딩 

변수별로 인코딩을 다르게 해준다. 

In [11]:
label_cols = ['Name', 'Ticket', 'Sex','Pclass','Embarked']
onehot_cols = [ 'Cabin',]
numerical_cols = [ 'Age', 'SibSp', 'Parch', 'Fare']

In [12]:
#라벨 인코딩 함수. c라는 매개변수를 받아서 맞게 트렌스폼 해준다. 
def label_encoder(c):
    le = LabelEncoder()
    return le.fit_transform(c)

In [13]:

#StandardScaler(): 평균을 제거하고 데이터를 단위 분산으로 조정한다. 
#그러나 이상치가 있다면 평균과 표준편차에 영향을 미쳐 변환된 데이터의 확산은 매우 달라지게 되는 함수
scaler = StandardScaler()

onehot_encoded_df = pd.get_dummies(all_df[onehot_cols])
label_encoded_df = all_df[label_cols].apply(label_encoder)
numerical_df = pd.DataFrame(scaler.fit_transform(all_df[numerical_cols]), columns=numerical_cols)
target_df = all_df[TARGET]

all_df = pd.concat([numerical_df, label_encoded_df,onehot_encoded_df, target_df], axis=1)
#all_df = pd.concat([numerical_df, label_encoded_df, target_df], axis=1)

## 모델링

In [14]:
drop_list=['Survived','Parch']

## not pseudo

In [15]:
train = all_df.iloc[:100000, :]#0개~100000개
test = all_df.iloc[100000:, :] #100000개~ 
#iloc은 정수형 인덱싱
test = test.drop('Survived', axis=1) #test에서 종속변수를 드랍한다. 
model_results = pd.DataFrame()
folds = 5

In [16]:
y= train.loc[:,'Survived']
X= train.drop(drop_list,axis=1)

## pycaret

In [17]:
caret_train=train.drop('Parch',axis=1)

In [18]:


!pip install pycaret==2.2.3

     |████████████████████████████████| 249 kB 866 kB/s 
     |████████████████████████████████| 102 kB 7.9 MB/s 
     |████████████████████████████████| 14.2 MB 6.4 MB/s 
     |████████████████████████████████| 167 kB 24.4 MB/s 
     |████████████████████████████████| 54 kB 1.6 MB/s 
     |████████████████████████████████| 79 kB 4.8 MB/s 
     |████████████████████████████████| 1.1 MB 25.7 MB/s 
     |████████████████████████████████| 1.7 MB 26.5 MB/s 
  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... - done
  Installing backend dependencies ... - \ | done
    Preparing wheel metadata ... - done
     |████████████████████████████████| 1.7 MB 29.0 MB/s 
  Created wheel for alembic: filename=alembic-1.4.1-py2.py3-none-any.whl size=158155 sha256=55daa34dfb9c7fe908c10616a504151353f0d3887f013d26df1568c88f0bc50d
  Stored in directory: /root/.cache/pip/wheels/be/5d/0a/9e13f53f4f5dfb67cd8d245bb7cdffe12f135846f491a283e3
  Created wheel

In [19]:
from pycaret.utils import version
import sklearn
print("pycaret version:", version())
print("sklearn version:", sklearn.__version__)

pycaret version: 2.2.3
sklearn version: 0.23.2


In [20]:
import pycaret.classification 
# from pycaret.regression import *
# reg1 = setup(data = train,target = 'Survived')
from pycaret.classification import *

In [21]:
caret_train.loc[:,'Embarked'].unique()
caret_train.loc[:,'Pclass'].unique()
caret_train.loc[:,'Sex'].unique()


array([1, 0])

In [22]:
category_caret={'Sex':['0','1'],'Pclass':['0','1','2'], 'Embarked':['0','1','2','3']}

In [23]:
setup(data = caret_train, 
      target = 'Survived',
      ordinal_features= category_caret,
      #numeric_imputation = 'Age','SibSp','Name','Ticket','Fare',
      fold=5,
      silent = True,
      session_id=1,
      #data_split_shuffle=True
      fold_shuffle=True

     )

,Description,Value
0,session_id,1
1,Target,Survived
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(100000, 18)"
5,Missing Values,False
6,Numeric Features,14
7,Categorical Features,3
8,Ordinal Features,True
9,High Cardinality Features,False


('80a2',
 {'acc': <pycaret.containers.metrics.classification.AccuracyMetricContainer at 0x7f1bf4376690>,
  'auc': <pycaret.containers.metrics.classification.ROCAUCMetricContainer at 0x7f1bf43766d0>,
  'recall': <pycaret.containers.metrics.classification.RecallMetricContainer at 0x7f1bf4376750>,
  'precision': <pycaret.containers.metrics.classification.PrecisionMetricContainer at 0x7f1bf43768d0>,
  'f1': <pycaret.containers.metrics.classification.F1MetricContainer at 0x7f1bf4376a50>,
  'kappa': <pycaret.containers.metrics.classification.KappaMetricContainer at 0x7f1bf4376c10>,
  'mcc': <pycaret.containers.metrics.classification.MCCMetricContainer at 0x7f1bf4376c50>},
 StratifiedKFold(n_splits=5, random_state=1, shuffle=True),
 5,
 'Survived',
 'lightgbm',
 None,
 {'USI',
  'X',
  'X_test',
  'X_train',
  '_all_metrics',
  '_all_models',
  '_all_models_internal',
  '_available_plots',
  '_gpu_n_jobs_param',
  '_internal_pipeline',
  '_ml_usecase',
  'create_model_container',
  'data_befo

In [24]:
#best_model = compare_models(sort = 'Accuracy', n_select = 4)

In [25]:
# gbc=create_model('gbc')

In [26]:
# gbc=tune_model(gbc
#               , optimize='AUC')

In [27]:
# print(gbc)

In [28]:
lightgbm=create_model('lightgbm',bagging_fraction=0.8, bagging_freq=5, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.9,
               importance_type='split', learning_rate=0.103, max_depth=-1,
               min_child_samples=30, min_child_weight=0.001, min_split_gain=0.4,
               n_estimators=40, n_jobs=-1, num_leaves=30, objective=None,
               random_state=1, reg_alpha=2, reg_lambda=0.2, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0
)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7793,0.8496,0.7382,0.7444,0.7413,0.5488,0.5489
1,0.7797,0.8492,0.7439,0.7424,0.7431,0.5503,0.5503
2,0.7858,0.8547,0.7489,0.7505,0.7497,0.5625,0.5625
3,0.7777,0.8493,0.7467,0.7376,0.7422,0.5468,0.5469
4,0.7825,0.8508,0.7429,0.7477,0.7453,0.5555,0.5555
Mean,0.7810,0.8507,0.7441,0.7445,0.7443,0.5528,0.5528
SD,0.0028,0.0021,0.0036,0.0044,0.0030,0.0056,0.0056


In [29]:
# lightgbm = tune_model(lightgbm
#                      , optimize='AUC' 
#                      )

In [30]:
print(lightgbm)

LGBMClassifier(bagging_fraction=0.8, bagging_freq=5, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.9,
               importance_type='split', learning_rate=0.103, max_depth=-1,
               min_child_samples=30, min_child_weight=0.001, min_split_gain=0.4,
               n_estimators=40, n_jobs=-1, num_leaves=30, objective=None,
               random_state=1, reg_alpha=2, reg_lambda=0.2, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)


In [31]:
cat_params={'depth': 9, 'l2_leaf_reg': 2, 'border_count': 254, 'verbose': False,
            'random_strength': 0.8, 'task_type': 'CPU', 'n_estimators': 270, 'random_state': 1}

In [32]:
catboost=create_model('catboost',**cat_params
                     )

In [33]:
# catboost = tune_model(catboost
#                      , optimize='AUC' 
#                      )

In [34]:
print(catboost.get_params())

{'depth': 9, 'l2_leaf_reg': 2, 'border_count': 254, 'verbose': False, 'random_strength': 0.8, 'task_type': 'CPU', 'n_estimators': 270, 'random_state': 1}


In [35]:
#ada=create_model('ada')

In [36]:
#ada=tune_model(ada,optimize='AUC')

In [37]:
test = all_df.iloc[100000:, :] #100000개~ 
X_test=test.drop(drop_list,axis=1)
X_test.head()

,Age,SibSp,Fare,Name,Ticket,Sex,Pclass,Embarked,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_X
100000,-0.937422,-0.539572,0.949786,10830,49,1,2,2,0,0,0,0,0,0,0,0,1
100001,1.123570,-0.539572,-1.273379,17134,49,0,2,2,0,0,0,0,0,0,0,0,1
100002,-0.937422,-0.539572,0.481059,9978,49,0,0,0,0,1,0,0,0,0,0,0,0
100003,-0.573717,-0.539572,-0.563310,13303,49,1,1,2,0,0,0,0,0,0,0,0,1
100004,-1.058657,-0.539572,0.125497,4406,49,0,0,0,0,1,0,0,0,0,0,0,0


### pycaret blending

In [38]:
blender_soft=blend_models(estimator_list=[lightgbm,catboost],
                          method='soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7809,0.8504,0.7439,0.7445,0.7442,0.5526,0.5526
1,0.7809,0.8496,0.7494,0.7417,0.7455,0.5531,0.5531
2,0.7856,0.8556,0.7514,0.7489,0.7502,0.5624,0.5624
3,0.7789,0.8499,0.7492,0.7385,0.7439,0.5494,0.5495
4,0.7831,0.8514,0.7467,0.7468,0.7468,0.5570,0.5570
Mean,0.7819,0.8514,0.7481,0.7441,0.7461,0.5549,0.5549
SD,0.0023,0.0022,0.0026,0.0037,0.0023,0.0044,0.0044


In [39]:
blender_hard=blend_models(estimator_list=[lightgbm,catboost],
                          method='hard')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7801,0.0000,0.7282,0.7509,0.7394,0.5492,0.5494
1,0.7796,0.0000,0.7334,0.7475,0.7403,0.5490,0.5491
2,0.7856,0.0000,0.7362,0.7568,0.7464,0.5608,0.5610
3,0.7773,0.0000,0.7309,0.7446,0.7377,0.5442,0.5443
4,0.7831,0.0000,0.7332,0.7537,0.7433,0.5555,0.5557
Mean,0.7811,0.0000,0.7324,0.7507,0.7414,0.5517,0.5519
SD,0.0029,0.0000,0.0027,0.0043,0.0031,0.0058,0.0058


### pycaret stacking

In [40]:
stacker=stack_models(estimator_list=[lightgbm,catboost],meta_model=lightgbm)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7809,0.8500,0.7464,0.7433,0.7448,0.5529,0.5529
1,0.7789,0.8489,0.7442,0.7409,0.7425,0.5488,0.5488
2,0.7854,0.8550,0.7477,0.7505,0.7491,0.5617,0.5617
3,0.7781,0.8497,0.7534,0.7352,0.7442,0.5483,0.5484
4,0.7843,0.8514,0.7384,0.7532,0.7457,0.5584,0.5585
Mean,0.7815,0.8510,0.7460,0.7446,0.7453,0.5540,0.5541
SD,0.0029,0.0021,0.0049,0.0065,0.0022,0.0053,0.0053


In [41]:
calibrated_blended = calibrate_model(blender_soft)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7804,0.8504,0.7385,0.7463,0.7424,0.5511,0.5511
1,0.7804,0.8496,0.7440,0.7435,0.7438,0.5517,0.5517
2,0.7856,0.8557,0.7474,0.7510,0.7492,0.5621,0.5621
3,0.7788,0.8500,0.7421,0.7417,0.7419,0.5483,0.5483
4,0.7836,0.8515,0.7440,0.7490,0.7465,0.5577,0.5577
Mean,0.7818,0.8514,0.7432,0.7463,0.7448,0.5542,0.5542
SD,0.0025,0.0022,0.0029,0.0034,0.0028,0.0050,0.0050


In [42]:
predictions = predict_model(calibrated_blended, data = X_test)
predictions.head()

,Age,SibSp,Fare,Name,Ticket,Sex,Pclass,Embarked,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_X,Label,Score
100000,-0.937422,-0.539572,0.949786,10830,49,1,2,2,0,0,0,0,0,0,0,0,1,0.0,0.8790
100001,1.123570,-0.539572,-1.273379,17134,49,0,2,2,0,0,0,0,0,0,0,0,1,1.0,0.5172
100002,-0.937422,-0.539572,0.481059,9978,49,0,0,0,0,1,0,0,0,0,0,0,0,1.0,0.8624
100003,-0.573717,-0.539572,-0.563310,13303,49,1,1,2,0,0,0,0,0,0,0,0,1,0.0,0.8092
100004,-1.058657,-0.539572,0.125497,4406,49,0,0,0,0,1,0,0,0,0,0,0,0,1.0,0.8384


In [43]:
predictions['Label']

100000    0.0
100001    1.0
100002    1.0
100003    0.0
100004    1.0
         ... 
199995    1.0
199996    0.0
199997    0.0
199998    1.0
199999    1.0
Name: Label, Length: 100000, dtype: object

In [44]:
# submission = pd.DataFrame(
#     {
#         'PassengerId': test_df["PassengerId"], 
#         'Survived': predictions['Label'],
#     })

In [45]:
print(submission)

       PassengerId  Survived
0           100000         1
1           100001         1
2           100002         1
3           100003         1
4           100004         1
...            ...       ...
99995       199995         1
99996       199996         1
99997       199997         1
99998       199998         1
99999       199999         1

[100000 rows x 2 columns]


In [46]:
#predictions['Label']=predictions['Label'].replace(float(0.0),int(0)).astype(np.int64)
#temp_predictions['Label']=predictions['Label'].map(lambda x: int(0) if x==(np.float(0.0)) else np.int(1))
#temp_predictions=pd.DataFrame(map(np.int,predictions['Label']))
temp_predictions=predictions['Label'].apply(pd.to_numeric).astype(int)

print(temp_predictions)

100000    0
100001    1
100002    1
100003    0
100004    1
         ..
199995    1
199996    0
199997    0
199998    1
199999    1
Name: Label, Length: 100000, dtype: int64


In [47]:
temp=temp_predictions.reset_index(drop=True)
#

print(temp)


0        0
1        1
2        1
3        0
4        1
        ..
99995    1
99996    0
99997    0
99998    1
99999    1
Name: Label, Length: 100000, dtype: int64


In [48]:
submission['Survived'] = temp
print(submission)

       PassengerId  Survived
0           100000         0
1           100001         1
2           100002         1
3           100003         0
4           100004         1
...            ...       ...
99995       199995         1
99996       199996         0
99997       199997         0
99998       199998         1
99999       199999         1

[100000 rows x 2 columns]


In [49]:
#submission['Survived'].astype(int)
submission.to_csv(f'pycaret_blend_submission_final.csv',index=False)

## Hard voting


In [50]:
# train_oof=np.zeros((100000,))
# train_oof=np.where()


## Soft Voting 

In [51]:
# def create_submission(model, test, test_passenger_id, model_name):
#     y_pred_test = model.predict_proba(test)[:, 1]
#     submission = pd.DataFrame(
#         {
#             'PassengerId': test_passenger_id, 
#             'Survived': (y_pred_test >= 0.5).astype(int),
#         }
#     )
#     submission.to_csv(f"submission_{model_name}.csv", index=False)
    
#     return y_pred_test

In [52]:
# test_pred_lightgbm = create_submission(
#     lightgbm, X_test, test_df["PassengerId"], "lightgbm"
# )
# # test_pred_ada = create_submission(
# #     ada, X_test, test_df["PassengerId"], "ada"
# # )
# test_pred_gbc = create_submission(
#      gbc, X_test, test_df["PassengerId"], "gbc"
# )
# test_pred_catboost = create_submission(
#     catboost, X_test, test_df["PassengerId"], "catboost"
# )

In [53]:
# test_pred_merged = (

#     test_pred_lightgbm + 
#     test_pred_catboost+
# #     test_pred_ada +
#      test_pred_gbc
# )

# test_pred_merged = np.round(test_pred_merged / 3)


In [54]:
# submission = pd.DataFrame(
#     {
#         'PassengerId': test_df["PassengerId"], 
#         'Survived': test_pred_merged.astype(int),
#     })
    

# submission.to_csv(f"submission_merged.csv", index=False)